In [15]:
import os
from src.config import NODE_TYPE

def load_result(path):
    results = {NODE_TYPE.PROCESS:set(), NODE_TYPE.NET:set(), NODE_TYPE.FILE: set()}
    with open(f'{path}', 'r') as logf:
        procs = logf.readline().strip()
        for proc in procs.split(','):
            if proc!='':
                    results[NODE_TYPE.PROCESS].add(proc)
        
        files = logf.readline().strip()
        for file in files.split(','):
            if file!='':
                    results[NODE_TYPE.FILE].add(file)
        
        nets = logf.readline().strip()
        for net in nets.split(','):
            if net!='':
                    results[NODE_TYPE.NET].add(net)
    return results[NODE_TYPE.FILE], results[NODE_TYPE.NET], results[NODE_TYPE.PROCESS] 


def load_label(data_name):
    attack_scene = {}
    with open(f'./label/{data_name}.txt', 'r') as f:
        while(1):
            line = f.readline()
            if not line:
                break

            if line[:6]=='ATTACK':
                t = line[6:].strip().split('/')
                # print(f'2018-{t[0]}-{t[1]}')
                attack_map = {NODE_TYPE.PROCESS:set(), NODE_TYPE.NET:set(), NODE_TYPE.FILE:set(),
                                'DAY':t[1]}
                
                while(1):
                    line = f.readline()
                    if line=='\n':
                        break
                    attack_map[NODE_TYPE.NET].add(line.strip())
                        
                while(1):
                    line = f.readline()
                    if line=='\n':
                        break
                    attack_map[NODE_TYPE.FILE].add(line.strip())

                while(1):
                    line = f.readline()
                    if line=='\n' or not line:
                        break
                    proc = line.strip()
                    attack_map[NODE_TYPE.PROCESS].add(proc)
                attack_scene[int(t[1])]=attack_map

    return attack_scene


def load_baseline_result(path):
    files = set()
    nets = set()
    processes = set()
    with open(f'{path}', 'r') as f:
        while(1):
            line = f.readline()
            if line=='\n':
                break
            files.add(line.strip())
        while(1):
            line = f.readline()
            if line=='\n':
                break
            nets.add(line.strip())
        while(1):
            line = f.readline()
            if not line:
                break
            if line=='\n':
                break
            processes.add(line.strip())
    # print(len(set(files)), len(set(nets)), len(processes))
    return files, nets, processes


In [50]:
root='./result/'
def print_TBII_DARPATC_lines(data_name, root='.'):
    lines = []
    labels = load_label(data_name)[12]
    labels[0] = labels[0]-set(['2710\tprofile (deleted)','2711\tprofile (deleted)', '2712\tprofile (deleted)'])
    for method in ['ThreaTrace', 'FLASH', 'MAGIC', 'PPIN']:
        line = f'{method:11}\t'
        results = {}
        if method=='PPIN':
            results[NODE_TYPE.FILE], results[NODE_TYPE.NET], results[NODE_TYPE.PROCESS] = load_result(f'{root}/{data_name}/12/log.txt')
            results[NODE_TYPE.PROCESS] = results[NODE_TYPE.PROCESS]-set(['2710\tprofile (deleted)','2711\tprofile (deleted)', '2712\tprofile (deleted)'])
        else:
            results[NODE_TYPE.FILE], results[NODE_TYPE.NET], results[NODE_TYPE.PROCESS] = load_baseline_result(f'{root}/baselines/{method.lower()}/{data_name}.txt')
        for nt in [NODE_TYPE.PROCESS,NODE_TYPE.FILE,NODE_TYPE.NET]:
            re = set(results[nt])
            ll = set(labels[nt])
            tp = len(re&ll)
            fp = len(re-ll)
            fn = len(ll-re)
            recall = tp/(tp+fn)
            precision = tp/(tp+fp)
            f1 = 2*recall*precision/(recall+precision)
            line+=f'{tp:4d}\t{fp:4d}\t{fn:4d}\t'
            line+=f'{recall:.2f}\t'
            line+=f'{precision:.2f}\t'
            line+=f'{f1:.2f}\t'
        lines.append(line)
    
    for line in lines:
        print(line)

def print_TBII_OPTC_lines(data_name, root='.'):
    lines = []
    
    labels = load_label('optc')[int(data_name)]
    for method in ['FLASH','PPIN']:
        line = f'{method:11}\t'
        results = {}
        if method=='PPIN':
            results[NODE_TYPE.FILE], results[NODE_TYPE.NET], results[NODE_TYPE.PROCESS] = load_result(f'{root}/optc/{data_name}/log.txt')
        else:
            results[NODE_TYPE.PROCESS], results[NODE_TYPE.FILE], results[NODE_TYPE.NET] = load_baseline_result(f'{root}/baselines/{method.lower()}/optc/{data_name}.txt')
            results[NODE_TYPE.FILE] = set([i.lower().replace('\\','/') for i in results[NODE_TYPE.FILE]])
        for nt in [NODE_TYPE.PROCESS,NODE_TYPE.FILE,NODE_TYPE.NET]:
            re = set(results[nt])
            ll = set(labels[nt])
            tp = len(re&ll)
            fp = len(re-ll)
            fn = len(ll-re)

            if tp>0:
                recall = tp/(tp+fn)
                precision = tp/(tp+fp)
                f1 = 2*recall*precision/(recall+precision)
                line+=f'{tp:4d}\t{fp:4d}\t{fn:4d}\t'
                line+=f'{recall:.2f}\t'
                line+=f'{precision:.2f}\t'
                line+=f'{f1:.2f}\t'
            else:
                line+=f'{tp:4d}\t{fp:4d}\t{fn:4d}\t'
                line+=f'  -\t  -\t  -\t'
        lines.append(line)
    
    for line in lines:
        print(line)

print('==========DARPA TC-CADETS===========')
print_TBII_DARPATC_lines('cadets3', root=root)
print('==========DARPA TC-THEIA===========')
print_TBII_DARPATC_lines('theia3', root=root)
print('==========OPTC-201===========')
print_TBII_OPTC_lines('0201', root=root)
print('==========OPTC-501===========')
print_TBII_OPTC_lines('0501', root=root)
print('==========OPTC-051===========')
print_TBII_OPTC_lines('0051', root=root)

==========DARPA TC-CADETS===========
ThreaTrace 	  10	  54	   8	0.56	0.16	0.24	   7	  83	   1	0.88	0.08	0.14	   7	   5	   4	0.64	0.58	0.61	
FLASH      	  13	  17	   5	0.72	0.43	0.54	   7	  55	   1	0.88	0.11	0.20	   7	   3	   4	0.64	0.70	0.67	
MAGIC      	   4	 141	  14	0.22	0.03	0.05	   1	  90	   7	0.12	0.01	0.02	   7	   5	   4	0.64	0.58	0.61	
PPIN       	  18	  25	   0	1.00	0.42	0.59	   8	  21	   0	1.00	0.28	0.43	  10	   1	   1	0.91	0.91	0.91	
==========DARPA TC-THEIA===========
ThreaTrace 	   5	   8	  57	0.08	0.38	0.13	   1	 812	   3	0.25	0.00	0.00	   2	   4	   3	0.40	0.33	0.36	
FLASH      	  22	   1	  40	0.35	0.96	0.52	   4	 952	   0	1.00	0.00	0.01	   5	  14	   0	1.00	0.26	0.42	
MAGIC      	  60	 146	   2	0.97	0.29	0.45	   4	  69	   0	1.00	0.05	0.10	   5	   4	   0	1.00	0.56	0.71	
PPIN       	  57	  36	   5	0.92	0.61	0.74	   4	  65	   0	1.00	0.06	0.11	   5	   0	   0	1.00	1.00	1.00	
==========OPTC-201===========
FLASH      	   9	  51	  11	0.45	0.15	0.23	   2	   9	   3	0.40	0.18	0.25	 

In [26]:
root='./result/'
def print_TBIV_lines(data_name, root='.'):
    attack_scene = load_label(data_name)
    lines = []
    for day in range(14):
        if not os.path.exists(f'{root}/{data_name}/{day}/log.txt'):
            continue
        day = int(day)
        results = {}
        results[NODE_TYPE.FILE], results[NODE_TYPE.NET], results[NODE_TYPE.PROCESS] = load_result(f'{root}/{data_name}/{day}/log.txt')
        
        
        if day not in attack_scene:
            line = f'{day}\t0\t{len(results[NODE_TYPE.PROCESS])}\t0\t-\t-\t-\t0\t{len(results[NODE_TYPE.FILE])}\t0\t-\t-\t-\t0\t{len(results[NODE_TYPE.NET])}\t0\t-\t-\t-\t'
        else:
            scene = attack_scene[day]
            line = f'{day}\t'
            for nt in [NODE_TYPE.PROCESS,NODE_TYPE.FILE,NODE_TYPE.NET]:
                re = results[nt]
                ll = scene[nt]
                tp = len(re&ll)
                fp = len(re-ll)
                fn = len(ll-re)
                pre = tp/(tp+fp)
                try:
                    rec = tp/(tp+fn)
                    f1 = 2*(pre*rec)/(pre+rec)
                except:
                        rec = 0
                        f1 = 0
                line += f'{tp}\t{fp}\t{fn}\t{rec:.2f}\t{pre:.2f}\t{f1:.2f}\t'
        
        lines.append(line)
    
    for line in lines:
        day = line.split('\t')[0]
        other = line[line.find('\t')+1:]
        line = ''
        if int(day) in attack_scene:
            line+= day+',\t'
        else:
            line+=day+',\t'
        line+=other.replace('\t', ',\t')
        line=line[:-2]

        print(line)

print('==========CADETS===========')
print_TBIV_lines('cadets3', root=root)
print('==========THEIA===========')
print_TBIV_lines('theia3', root=root)

==========CADETS===========
6,	2,	3,	0,	1.00,	0.40,	0.57,	3,	4,	0,	1.00,	0.43,	0.60,	5,	1,	0,	1.00,	0.83,	0.91
8,	0,	1,	0,	-,	-,	-,	0,	2,	0,	-,	-,	-,	0,	0,	0,	-,	-,	-
11,	1,	3,	0,	1.00,	0.25,	0.40,	2,	4,	1,	0.67,	0.33,	0.44,	3,	1,	0,	1.00,	0.75,	0.86
12,	18,	25,	0,	1.00,	0.42,	0.59,	8,	21,	0,	1.00,	0.28,	0.43,	10,	1,	1,	0.91,	0.91,	0.91
13,	4,	4,	0,	1.00,	0.50,	0.67,	6,	7,	0,	1.00,	0.46,	0.63,	5,	1,	0,	1.00,	0.83,	0.91
==========THEIA===========
10,	46,	1,	3,	0.94,	0.98,	0.96,	3,	6,	0,	1.00,	0.33,	0.50,	2,	0,	2,	0.50,	1.00,	0.67
11,	12,	0,	0,	1.00,	1.00,	1.00,	0,	1,	0,	0.00,	0.00,	0.00,	2,	0,	0,	1.00,	1.00,	1.00
12,	60,	36,	5,	0.92,	0.62,	0.75,	4,	65,	0,	1.00,	0.06,	0.11,	5,	0,	0,	1.00,	1.00,	1.00
